In [1]:
#the purpose of this notebook is to review machine learning before my IMH interview

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import naive_bayes

from matplotlib import pyplot as plt
%matplotlib inline

In [32]:
#read in the titanic data
train = pd.read_csv("titanic train.csv")
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [33]:
#drop some of the columns that will be hard to work with
train.small = train.drop(["Name","PassengerId","SibSp","Ticket","Cabin", \
           "Embarked"],axis = 1)
train.small.head()

,Survived,Pclass,Sex,Age,Parch,Fare
0,0,3,male,22.0,0,7.2500
1,1,1,female,38.0,0,71.2833
2,1,3,female,26.0,0,7.9250
3,1,1,female,35.0,0,53.1000
4,0,3,male,35.0,0,8.0500


In [34]:
#check for missing values
train.small.columns[train.small.isnull().any()]

Index(['Age'], dtype='object')

In [35]:
#replace missing values with median for simplicity's sake
mean_age = np.mean(train['Age'])
train.clean = train.small.fillna(mean_age)

In [36]:
#check for missing values again
train.clean.columns[train.clean.isnull().any()]

Index([], dtype='object')

In [37]:
#change character values to numbers
train.clean = train.clean.replace(['female', 'male'], [0, 1])
train.clean.head(5)

,Survived,Pclass,Sex,Age,Parch,Fare
0,0,3,1,22.0,0,7.2500
1,1,1,0,38.0,0,71.2833
2,1,3,0,26.0,0,7.9250
3,1,1,0,35.0,0,53.1000
4,0,3,1,35.0,0,8.0500


In [40]:
train.clean = pd.get_dummies(train.clean,columns = ["Pclass"])
train.clean.head()

,Survived,Sex,Age,Parch,Fare,Pclass_1,Pclass_2,Pclass_3
0,0,1,22.0,0,7.2500,0.0,0.0,1.0
1,1,0,38.0,0,71.2833,1.0,0.0,0.0
2,1,0,26.0,0,7.9250,0.0,0.0,1.0
3,1,0,35.0,0,53.1000,1.0,0.0,0.0
4,0,1,35.0,0,8.0500,0.0,0.0,1.0


In [ ]:
#lets fit it to a naive bayes model first


In [41]:
#now that the data is clean
#we will fit a Random Forest model to it
clf = RandomForestClassifier(n_estimators=100,random_state=0,n_jobs = -1)

#split up the data into predictors and response
X = train.clean.drop(['Survived'], axis = 1)
Y = train.clean['Survived']

clf.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [46]:
#run I clean up the test data, predict with my forest,
#and then save the results to a csv file
test = pd.read_csv("titanic test.csv")
test.small = test.drop(["Name","PassengerId","SibSp","Ticket","Cabin", \
           "Embarked"],axis = 1)
test.clean = test.small.fillna(mean_age)
test.clean = test.clean.replace(['female', 'male'], [0, 1])
test.clean = pd.get_dummies(test.clean,columns = ["Pclass"])
test.clean.head(5)

,Sex,Age,Parch,Fare,Pclass_1,Pclass_2,Pclass_3
0,1,34.5,0,7.8292,0.0,0.0,1.0
1,0,47.0,0,7.0000,0.0,0.0,1.0
2,1,62.0,0,9.6875,0.0,1.0,0.0
3,1,27.0,0,8.6625,0.0,0.0,1.0
4,0,22.0,1,12.2875,0.0,0.0,1.0


In [48]:
output = clf.predict(test.clean)
# print(output)

In [50]:
#leave this as is and run it
outcome_data = test.drop(['Name', 'Ticket', 'Cabin','Fare', 'Embarked', 'Age','Sex', 'Pclass','Pclass','SibSp', 'Parch'], axis = 1)
outcome_data["Survived"] = output
outcome_data.head(5)

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [52]:
#write to csv
outcome_data.to_csv('RF-Matt.csv',index = False)